# Amazon Bedrock Application Inference Profile

이 노트북은 조직이 Amazon Bedrock 애플리케이션 추론 프로파일을 구현, 테스트, 검증 및 운영하는 방법을 보여줍니다. 애플리케이션 추론 프로파일을 효과적으로 관리하고 활용하는 방법에 대한 포괄적인 이해를 제공하는 것이 목표입니다.

# Overview

아마존 베드락 애플리케이션 추론 프로파일을 통해 조직은 모든 베드락 기반 파운데이션 모델에 비용 할당 태그를 지정하여 비용 센터, 사업 단위, 팀, 애플리케이션과 같은 조직 분류 체계별로 사용량을 분류할 수 있습니다. 여러 워크로드에서 AI 지출을 관리하는 이 확장 가능한 프로그래밍 방식은 수동 프로세스에 대한 의존도를 낮추고 비용 초과 위험을 줄이며 중요한 애플리케이션이 우선순위를 갖도록 보장합니다. AI 관련 비용에 대한 향상된 가시성과 제어 기능을 통해 조직은 GenAI 투자를 최적화하고 보다 효율적으로 혁신을 추진할 수 있습니다. 이 노트북은 애플리케이션 추론 프로필을 생성하고 모델을 호출하는 데 사용하는 방법을 보여줍니다.

# Architecture
![Architecture](./images/architecture.png)

Amazon Bedrock의 추론 프로파일은 두 가지 유형이 있습니다:
1. 크로스 리전 추론 프로파일
- Bedrock 서비스가 사전 정의
- 여러 리전으로 모델 요청을 라우팅
- 복원력 향상과 처리량 개선
- 트래픽 급증 관리에 효과적

2. 애플리케이션 추론 프로파일
- 사용자가 직접 생성
- 단일 리전 또는 여러 리전으로 라우팅 가능
- 비용과 모델 사용량 추적

주요 이점:
- CloudWatch 로그를 통한 사용량 지표 추적
- 태그 기반 비용 모니터링
- 크로스 리전 추론으로 처리량 향상

이 노트북은 애플리케이션 추론 프로파일을 생성하고 이를 모델 호출에 활용하는 방법을 보여줍니다.

# Use case

이 노트북에는 다음과 같은 다양한 Amazon Bedrock 애플리케이션 추론 프로파일 API 기능에 대한 구현, 테스트 및 유효성 검사 단계가 포함되어 있습니다:
* Create Inference Profile
* Get Inference Profile
* List Inference Profiles
* Invoke model with Inference Profile using Converse API
* Invoke model with Inference Profile using ConverseStream API
* Invoke model with Inference Profile using InvokeModel API
* Invoke model with Inference Profile using InvokeModelWithResponseStream API
* Tag Resource
* List Tags For Resource
* Untag Resource
* Cleanup


## Prerequisites

1. AWS 계정이 생성되었는지 확인합니다.: [Link](https://docs.aws.amazon.com/accounts/latest/reference/manage-acct-creating.html)
2. 사용자에게 Amazon Bedrock에서 올바른 모델에 액세스할 수 있는 권한이 있는지 확인합니다.: [Link](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html)
3. 이 노트북은 AWS 자격 증명을 설정하여 진행했습니다.

## Notebook Code with Comments

### Install dependecies

In [ ]:
!pip install --upgrade --force-reinstall boto3 botocore awscli

In [ ]:
import boto3
print(boto3.__version__)

In [ ]:
import boto3
import sagemaker
import re
import time
import json

session = boto3.Session()
sagemaker_session = sagemaker.Session()
#role = sagemaker.get_execution_role()
current_region = sagemaker_session.boto_region_name 

bedrock = session.client("bedrock", region_name=current_region)
br = session.client("bedrock-runtime", region_name=current_region)

In [ ]:
current_region

### List application inference profiles

In [ ]:
bedrock.list_inference_profiles(typeEquals='APPLICATION')

### List cross region inference profiles

In [ ]:
bedrock.list_inference_profiles(typeEquals='SYSTEM_DEFINED')

In [ ]:
models = [{'modelArn': fm['modelArn'], 'modelName':fm['modelName'], 'modelId': fm['modelId']} for fm in bedrock.list_foundation_models()['modelSummaries'] if fm['modelName'].startswith('Nova') and fm['modelLifecycle']['status']== 'ACTIVE']
models

## Setup single Region application inference profile

단일 리전에 대한 애플리케이션 추론 프로필을 생성하려면 기반 모델의 ARN을 지정해야 합니다. 해당 리전의 모델에 대한 요청의 사용량과 비용이 추적됩니다. 요청 생성 시 다음 매개변수를 제공해야 합니다:

* inferenceProfileName - 추론 프로필의 이름
* modelSource - 단일 리전의 경우 copyFrom 속성에 기반 모델의 ARN을 지정
* description - 추론 프로필에 대한 설명 (선택사항)
* tags - 추론 프로필에 태그 첨부. AWS 비용 할당 태그를 사용하여 비용 추적 가능. 프로젝트 ID, 부서 ID 또는 원하는 방식으로 비용 추적 가능

In [ ]:
#Amazon Nova Pro
#Model ARN: arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-pro-v1:0
#Model ID: amazon.nova-pro-v1:0

modelId = 'amazon.nova-pro-v1:0'
modelARN = [m['modelArn'] for m in models if m['modelId'] == modelId][0]
modelId, modelARN 

In [8]:
inf_profile_response = bedrock.create_inference_profile(
    inferenceProfileName='NovaProPart',
    description='Application profile for Nova Pro',
    modelSource={
        'copyFrom': modelARN
    },
    tags=[
        {
            'key': 'projectId',
            'value': 'parts2025'
        },
    ]
)

In [ ]:
inf_profile_arn = inf_profile_response['inferenceProfileArn']
inf_profile_arn

In [ ]:
bedrock.get_inference_profile(inferenceProfileIdentifier=inf_profile_arn)

In [ ]:
inf_profile_id = bedrock.get_inference_profile(inferenceProfileIdentifier=inf_profile_arn)['inferenceProfileId']
inf_profile_id

### Example usage with Converse API

추론 프로파일을 사용하려면 modelId 필드에 추론 프로파일의 ARN을 지정합니다. 

In [ ]:
from time import time
from pprint import pprint
system_prompt = "You are an expert on AWS services and always provide correct and concise answers."
input_message = "비용 효율적인 애플리케이션을 구성하기 위해서는 Amazon S3 또는 EFS에 문서를 저장하는게 좋을까요??"
start = time()
response = br.converse(
    modelId=inf_profile_arn,
    system=[{"text": system_prompt}],
    messages=[{
        "role": "user",
        "content": [{"text": input_message}]
    }]
)
end = time()

print(f"Response time: {int(end-start)} second(s)")
print(f"Using Application Inf Profile::Response output: {response['output']['message']['content']}")


In [ ]:
import re

text = response['output']['message']['content'][0]['text']
clean_text = re.sub(r'\*\*(.*?)\*\*', r'\1', text)
formatted_text = clean_text.strip()
print(formatted_text)

### Example usage with invoke model

In [ ]:
import json
system_list = [
            {
                "text": "You are an expert on AWS services and always provide correct and concise answers"
            }
]
message_list = [{"role": "user", "content": [{"text": "비용 효율적인 애플리케이션을 구성하기 위해서는 Amazon S3 또는 EFS에 문서를 저장하는게 좋을까요??"}]}]
inf_params = {"max_new_tokens": 500, "top_p": 0.9, "top_k": 20, "temperature": 0.7}

body = json.dumps({
    "schemaVersion": "messages-v1",
    "system": system_list,
    "messages":message_list,
    "inferenceConfig" : inf_params,
}
)

accept = 'application/json'
contentType = 'application/json'
start = time()
response = br.invoke_model(body=body, modelId=inf_profile_arn, accept=accept, contentType=contentType)
end = time()
response_body = json.loads(response.get('body').read())

print(f"Response time: {int(end-start)} second(s)")
print(f"Using Application Inf Profile::Response output: {response_body['output']['message']['content']}")

In [ ]:
import re

text = response_body['output']['message']['content'][0]['text']
clean_text = re.sub(r'\*\*(.*?)\*\*', r'\1', text)
formatted_text = clean_text.strip()
print(formatted_text)

## 다중 리전 애플리케이션 추론 프로파일
리전 간 애플리케이션 추론 프로파일을 생성하려면 크로스 리전 추론 프로파일의 ARN을 지정하고, 나머지 매개변수는 단일 리전 애플리케이션 추론 프로파일과 동일합니다.

* inferenceProfileName - 추론 프로파일의 이름
* modelSource - 다중 리전 애플리케이션 프로파일의 경우 copyFrom 속성에 크로스 리전(시스템 정의) 추론 프로파일의 ARN을 지정
* description - 추론 프로파일에 대한 설명 (선택사항)
* tags - 추론 프로파일에 태그 첨부. AWS 비용 할당 태그를 사용하여 비용 추적 가능. 프로젝트 ID, 부서 ID 또는 원하는 방식으로 비용 추적 가능

In [ ]:
cr_inf_profile = [ip for ip in bedrock.list_inference_profiles(typeEquals='SYSTEM_DEFINED')['inferenceProfileSummaries'] if ip['inferenceProfileName'] == 'US Nova Pro'][0]
cr_inf_profile

In [27]:
cr_inf_profile_arn = 'arn:aws:bedrock:us-west-2::foundation-model/amazon.nova-pro-v1:0'
cr_app_inf_profile_response = bedrock.create_inference_profile(
    inferenceProfileName='NovaProSupplyCrossRegion',
    description='Application profile for Nova Pro with cross regional routing',
    modelSource={
        'copyFrom': cr_inf_profile_arn
    },
    tags=[
        {
            'key': 'projectId',
            'value': 'supply2026'
        },
    ]
)

In [ ]:
cr_app_inf_profile_arn = cr_app_inf_profile_response['inferenceProfileArn']
cr_app_inf_profile_arn

In [ ]:
bedrock.get_inference_profile(inferenceProfileIdentifier=cr_app_inf_profile_arn)

### Example usage Converse API

In [ ]:
from time import time
system_prompt = "You are an expert on AWS services and always provide correct and concise answers."
input_message = "비용 효율적인 애플리케이션을 구성하기 위해서는 Amazon S3 또는 EFS에 문서를 저장하는게 좋을까요?"

start = time()
response = br.converse(
    modelId=cr_app_inf_profile_arn,
    system=[{"text": system_prompt}],
    messages=[{
        "role": "user",
        "content": [{"text": input_message}]
    }]
)
end = time()
print(f"Response time: {int(end-start)} second(s)")
print(f"Using Application Inf Profile::Response output: {response['output']['message']['content']}")

In [ ]:
import re

text = response['output']['message']['content'][0]['text']
clean_text = re.sub(r'\*\*(.*?)\*\*', r'\1', text)
formatted_text = clean_text.strip()
print(formatted_text)

### Example usage Invoke Model

In [ ]:
import json
system_list = [
            {
                "text": "You are an expert on AWS services and always provide correct and concise answers"
            }
]
message_list = [{"role": "user", "content": [{"text": "비용 효율적인 애플리케이션을 구성하기 위해서는 Amazon S3 또는 EFS에 문서를 저장하는게 좋을까요??"}]}]
inf_params = {"max_new_tokens": 500, "top_p": 0.9, "top_k": 20, "temperature": 0.7}

body = json.dumps({
    "schemaVersion": "messages-v1",
    "system": system_list,
    "messages":message_list,
    "inferenceConfig" : inf_params,
}
)
accept = 'application/json'
contentType = 'application/json'
start = time()
response = br.invoke_model(body=body, modelId=cr_app_inf_profile_arn, accept=accept, contentType=contentType)
end = time()
response = json.loads(response.get('body').read())

print(f"Response time: {int(end-start)} second(s)")
print(f"Using Application Inf Profile::Response output: {response['output']['message']['content']}")



In [ ]:
import re

text = response['output']['message']['content'][0]['text']
clean_text = re.sub(r'\*\*(.*?)\*\*', r'\1', text)
formatted_text = clean_text.strip()
print(formatted_text)

### Delete Inference Profile

In [ ]:
app_inf_profiles = bedrock.list_inference_profiles(typeEquals='APPLICATION')['inferenceProfileSummaries']
app_inf_profiles

In [ ]:
for app_ip in app_inf_profiles:
    response = bedrock.delete_inference_profile(inferenceProfileIdentifier=app_ip['inferenceProfileArn'])

In [ ]:
app_inf_profiles = bedrock.list_inference_profiles(typeEquals='APPLICATION')['inferenceProfileSummaries']
app_inf_profiles

# END OF NOTEBOOK 

## 관련 문서 
- https://community.aws/content/2qB7GuzG89Dgz3kGvCpeyk2UU6o/application-inference-profiles-for-aws-bedrock-foundation-model-cost-and-access-management
- https://community.aws/content/2oa6QAE7tgxmAsgAExXY2B2MqCV/getting-started-with-bedrock-application-inference-profile?lang=en
- https://github.com/GopiKWork/amazon-bedrock/blob/main/inference-profiles/application_inference_profile.ipynb
- https://aws.amazon.com/blogs/machine-learning/track-allocate-and-manage-your-generative-ai-cost-and-usage-with-amazon-bedrock/
- https://github.com/aws-samples/amazon-bedrock-samples/blob/main/poc-to-prod/inference-profiles/inference-profile-basics.ipynb